# DAC配置与DP2031测试流程概览

- 目标：依次修改 `DACCONFIG.txt` 配置、下发到串口并记录DP2031的VDC3电流。
- 环境：需要正确安装 PyVISA、pyserial、pandas、matplotlib，并确认DP2031的VISA地址与串口号。
- 流程：连接设备 → 设置电源输出 → 循环修改/下发DAC配置 → 记录并可视化电流数据。
- 注意：运行前请先在代码中更新串口号与DP2031地址，避免默认值导致连接失败。

## 1. 导入必要的库和辅助函数


In [10]:
import time
import os
import serial
import serial.tools.list_ports
import pyvisa
import pandas as pd
import matplotlib.pyplot as plt
from modify_dacconfig import modify_dac_config


In [11]:
import serial
print(serial.__file__)

D:\SVN\usr\weihaoyu\Rigel\cp_test\cp_test\Lib\site-packages\serial\__init__.py


In [12]:
from config_loader import _calculate_gear, _voltage_to_dac_code, load_config_data, generate_full_config_commands

In [13]:
from serial_controller import SerialController

## 2. 连接DP2031电源

使用PyVISA连接到DP2031电源。请将 `dp2031_visa_address` 替换为您的实际VISA地址。

---



In [14]:
rm = pyvisa.ResourceManager()
print(rm.list_resources())

('USB0::0x1AB1::0xA4A8::DP2A265201089::INSTR', 'TCPIP0::172.18.26.76::inst0::INSTR', 'TCPIP0::K-34465A-05164.local::hislip0::INSTR', 'TCPIP0::K-34465A-05164.local::inst0::INSTR', 'TCPIP0::172.18.26.44::inst0::INSTR', 'TCPIP0::172.18.26.142::inst0::INSTR', 'TCPIP0::172.18.26.92::inst0::INSTR', 'TCPIP0::172.18.26.108::inst0::INSTR', 'ASRL1::INSTR')


In [15]:

dp2031 = None

dp2031_visa_address = "USB0::0x1AB1::0xA4A8::DP2A265201089::INSTR"  # 替换为您的DP2031的VISA地址

try:
    dp2031 = rm.open_resource(dp2031_visa_address)
    print(f"成功连接到DP2031: {dp2031.query('*IDN?')}")
    # dp2031.write("*RST") # 重置电源到已知状态
    time.sleep(0.1)
except Exception as e:
    print(f"连接DP2031失败: {e}")


成功连接到DP2031: Rigol Technologies,DP2031,DP2A265201089,00.01.01.00.22



In [16]:
# --- DP2031 电源控制逻辑 ---


try:
    dp2031 = rm.open_resource(dp2031_visa_address)
    print(f"成功连接到DP2031: {dp2031.query('*IDN?')}")
    # dp2031.write("*RST") # 重置电源到已知状态
    # time.sleep(1)

    # 设置VDC3 (假设对应CH2) 为1.9V, 0.5A 限流，并开启输出
    vdc3_channel = 2 # VDC3 对应的通道号
    vdc3_voltage = 1.9
    vdc3_current_limit = 0.5
    dp2031.write(f'APPL CH{vdc3_channel},{vdc3_voltage},{vdc3_current_limit}')
    dp2031.write(f'OUTP CH{vdc3_channel},ON')
    print(f"已将VDC3 (CH{vdc3_channel}) 配置为 {vdc3_voltage}V, {vdc3_current_limit}A 并开启输出。")
    time.sleep(1) # 等待稳定

except Exception as e:
    print(f"连接DP2031失败或配置VDC3失败: {e}")


成功连接到DP2031: Rigol Technologies,DP2031,DP2A265201089,00.01.01.00.22

已将VDC3 (CH2) 配置为 1.9V, 0.5A 并开启输出。


## 3. 串口下发与电流记录流程

- 在代码中先设置 `port_to_connect`（例如 `COM3`），再运行本单元。
- 循环 1~7 次，每次调用 `modify_dac_config` 生成新的 `DACCONFIG.txt`，随后通过串口发送完整配置。
- 每轮结束后从 DP2031 读取 VDC3 (CH2) 电流，并累计到 `vdc3_current_readings`。
- 如果需要中断，可在执行时按 `Ctrl + C`。脚本末尾会自动断开串口并关闭 VDC3 输出。



In [17]:
# --- 主执行逻辑 ---

serial_controller = SerialController()
vdc3_current_readings = [] # 用于存储VDC3的电流读数

# 替换为你的实际串口号，例如 'COM3' 或 '/dev/ttyUSB0'
# 可以通过运行 serial_controller.scan_ports() 来查看可用串口

port_to_connect = "COM3"  # <--- 请在这里修改为你的串口号

success, message = serial_controller.connect(port_to_connect)
if not success:
    print(f"串口连接失败: {message}")
else:
    print("串口连接成功。开始循环配置...")
    
    try:
        for iteration_index in range(1, 8):  # 循环 DAC1 到 DAC7
            print(f"\n--- 处理 DAC{iteration_index} ---")
            modify_dac_config(iteration_index)
            time.sleep(1) # 等待文件系统写入完成

            print(f"\n--- 执行第 {iteration_index} 次DAC配置 ---")
            filepath = "DACCONFIG.txt"
            try:
                dac_status = load_config_data(filepath)
                commands_generator = generate_full_config_commands(dac_status)
                for command, delay in commands_generator:
                    serial_controller.send_command(command)
                    time.sleep(delay)
                print(f"第 {iteration_index} 次DAC配置完成！")

                # 记录VDC3端口的电流（pyvisa对象无connected属性，直接判空）
                if dp2031 is not None:
                    current_vdc3 = float(dp2031.query(f'MEAS:CURR? CH{vdc3_channel}')) # vdc3_channel 在DP2031连接单元格中定义
                    vdc3_current_readings.append(current_vdc3)
                    print(f"VDC3 (CH{vdc3_channel}) 当前电流: {current_vdc3:.4f} A")
                else:
                    print("DP2031未连接或未启用，无法记录VDC3电流。")

            except Exception as e:
                print(f"DAC配置失败: {e}")
                
            if iteration_index < 7: # 最后一次循环后不再等待
                print(f"等待5秒进行下一次修改和配置...\n")
                time.sleep(5) # 等待5秒
            
    except KeyboardInterrupt:
        print("用户中断循环。")
    except Exception as e:
        print(f"主循环发生错误: {e}")
    finally:
        serial_controller.disconnect()
        print("脚本执行完毕，串口已断开。")
        if dp2031:
            try:
                dp2031.write(f'OUTP CH{vdc3_channel},OFF') # 关闭VDC3输出
                dp2031.close()
                print("DP2031已断开连接，VDC3通道已关闭。")
            except Exception as e:
                print(f"断开DP2031或关闭VDC3时发生错误: {e}")

print("VDC3电流记录:", vdc3_current_readings)


串口连接失败: 无法打开串口 COM3
错误: could not open port 'COM3': FileNotFoundError(2, '系统找不到指定的文件。', None, 2)
VDC3电流记录: []


## 4. 保存与可视化

- 默认在当前目录下创建 `results/`，文件名以输入的后缀区分。
- 若 `vdc3_current_readings` 为空，则跳过保存和绘图（常见于串口/电源未连好）。
- 建议在完成多轮配置后再运行本单元，以便生成完整的CSV与PNG。

In [19]:
# --- 新增：创建结果文件夹与获取文件名 ---
results_dir = "results"
if not os.path.exists(results_dir):
    os.makedirs(results_dir)  # 如果文件夹不存在则创建

file_suffix = input("请输入本次测试的文件名后缀 (例如 test_01): ")
if not file_suffix:
    file_suffix = "default"

# 定义包含文件夹路径的完整文件名
log_path = os.path.join(results_dir, f"vdc3_current_log_{file_suffix}.csv")
png_path = os.path.join(results_dir, f"vdc3_current_plot_{file_suffix}.png")

print(f"日志将保存至: {log_path}")
print(f"图片将保存至: {png_path}")
# ------------------------------------


# --- 数据保存与可视化 ---

if vdc3_current_readings:
    # 保存为CSV文件
    df_current = pd.DataFrame({'Iteration': range(1, len(vdc3_current_readings) + 1), 
                               'VDC3 Current (A)': vdc3_current_readings})
    df_current.to_csv(log_path, index=False)
    print(f"VDC3电流数据已保存到 '{log_path}'")

    # 绘制电流变化图
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(vdc3_current_readings) + 1), vdc3_current_readings, 
             marker='o', linestyle='-', color='blue', label='VDC3 Current')
    plt.title("VDC3 Current During DAC Configuration")
    plt.xlabel("DAC Iteration (1-7)")
    plt.ylabel("Current (A)")
    plt.grid(True)
    plt.legend()
    plt.xticks(range(1, 8)) # 显示1-7的刻度
    plt.savefig(png_path)
    print(f"VDC3电流变化图已保存为 '{png_path}'")
    plt.show()
else:
    print("没有VDC3电流数据可供保存和绘制。")


请输入本次测试的文件名后缀 (例如 test_01):  


日志将保存至: results\vdc3_current_log_default.csv
图片将保存至: results\vdc3_current_plot_default.png
没有VDC3电流数据可供保存和绘制。
